In [ ]:
!pip install -q bitsandbytes datasets accelerate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (py

In [ ]:
import os

import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

model = AutoModelForCausalLM.from_pretrained("aaditya/Llama3-OpenBioLLM-8B", quantization_config=BitsAndBytesConfig(load_in_8bit=True))

tokenizer = AutoTokenizer.from_pretrained("aaditya/Llama3-OpenBioLLM-8B")

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6815744 || all params: 8037076992 || trainable%: 0.08480376642881861


In [ ]:
import transformers
from datasets import load_dataset

data = load_dataset('lighteval/med_dialog', 'healthcaremagic')


In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['tgt', 'src', 'id'],
        num_rows: 181122
    })
    validation: Dataset({
        features: ['tgt', 'src', 'id'],
        num_rows: 22641
    })
    test: Dataset({
        features: ['tgt', 'src', 'id'],
        num_rows: 22642
    })
})

In [ ]:
# Reduce GPU memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# Clear any existing cache
torch.cuda.empty_cache()

# Load dataset and define tokenization
data = load_dataset('lighteval/med_dialog', 'healthcaremagic')

def tokenize_function(samples):
    inputs = [f"{src} {tokenizer.sep_token} {tgt}" for src, tgt in zip(samples['src'], samples['tgt'])]
    return tokenizer(inputs, truncation=True, padding="max_length", max_length=256)  # Reduced sequence length

tokenized_data = data.map(tokenize_function, batched=True, remove_columns=['tgt', 'src', 'id'])


Map:   0%|          | 0/181122 [00:00<?, ? examples/s]

In [ ]:
import torch
torch.cuda.empty_cache()


In [ ]:

# Enable gradient checkpointing
model.gradient_checkpointing_enable()
model.config.use_cache = False

# Initialize Trainer with modified TrainingArguments
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    args=transformers.TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    warmup_steps=100,
    max_steps=50,
    learning_rate=1e-4,  # Lower learning rate
    fp16=True,
    logging_steps=1,
    output_dir="outputs",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)



# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,3.397900
2,3.432300
3,3.528300
4,3.540900
5,3.412700
6,3.339100
7,3.484300
8,3.184900
9,3.470400
10,3.217800


TrainOutput(global_step=50, training_loss=3.2151490831375122, metrics={'train_runtime': 1543.0618, 'train_samples_per_second': 0.518, 'train_steps_per_second': 0.032, 'total_flos': 9230426623180800.0, 'train_loss': 3.2151490831375122, 'epoch': 0.004416912357416548})